In [ ]:
# q_vis_response_plotter.ipynb
# Avery Krieger 05/23/2023

# Adapts code from uniform_flash.py to analyize and plot the AstA-->Chrimson, GCaMP data

%load_ext autoreload
%autoreload 2

from visanalysis.analysis import imaging_data, shared_analysis
from visanalysis.util import plot_tools

from matplotlib import pyplot as plt
from matplotlib.ticker import FixedLocator, FixedFormatter

import os
import numpy as np
from pathlib import Path

# Load in flies
folder_path = "/Volumes/ABK2TBData/data_repo/bruker/20230519.selected"
exp_date = "2023-05-19"

# asta_fly1_prox = [folder_path, exp_date, "2", "medulla_terminals"]
asta_fly1_prox = [folder_path, exp_date, "1", "medulla_terminals_cat2"]

asta_fly2_prox_all = [folder_path, exp_date, "5", "medulla_terminals_all"]
asta_fly2_prox_cat2 = [folder_path, exp_date, "5", "medulla_terminals_cat2"]

asta_fly3_prox_all = [folder_path, exp_date, "8", "medulla_terminals_all"]

asta_prox_all = [asta_fly1_prox, asta_fly2_prox_cat2, asta_fly3_prox_all]
asta_fly_list = [1, 2, 3]

# Housekeeping:
condition_name = 'current_led_intensity'
save_directory = "/Volumes/ABK2TBData/lab_repo/analysis/outputs/opto_step_series/"
Path(save_directory).mkdir(exist_ok=True)